In [1]:
import time
import numpy as np
import random
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical

In [2]:
print(str(time.ctime()) + ": Initializing...")

dset_train = np.load('/gpfs/alpine/proj-shared/gen150/junqi/hea/without-channel/HEA_train.npy')
dset_val = np.load('/gpfs/alpine/proj-shared/gen150/junqi/hea/without-channel/HEA_val.npy')
lt = np.load('/gpfs/alpine/gen150/proj-shared/junqi/hea/label_T_train.npy')
lv = np.load('/gpfs/alpine/gen150/proj-shared/junqi/hea/label_T_val.npy')

print(str(time.ctime()) + ": Successfully loaded all data sets!")

Tue Jul  6 14:30:48 2021: Initializing...
Tue Jul  6 14:30:50 2021: Successfully loaded all data sets!


In [3]:
train_size = 4000
val_size = 1000
dim = 64
hea_size = 40

In [4]:
trainset = dset_train[0:train_size, 0:hea_size, 0:hea_size, 0:hea_size]
valset = dset_val[0:val_size, 0:hea_size, 0:hea_size, 0:hea_size]

label_training = lt[0:train_size]
label_validation = lv[0:val_size]

In [5]:
print(dset_train.shape)
print(trainset.shape)
print(dset_val.shape)
print(valset.shape)

(40000, 46, 46, 46)
(4000, 40, 40, 40)
(10000, 46, 46, 46)
(1000, 40, 40, 40)


In [6]:
train_2D = np.reshape(trainset, (trainset.shape[0], -1))
val_2D = np.reshape(valset, (valset.shape[0], -1))
print(train_2D.shape)
print(val_2D.shape)

(4000, 64000)
(1000, 64000)


In [7]:
train_nonzero = np.ndarray(shape=(train_size, dim))
for row in range(train_2D.shape[0]):
    mat = train_2D[row].flatten()
    mat = mat[np.where(mat != 0)]
    train_nonzero[row, :] = mat[0:dim]
print(train_nonzero.shape)
print(train_nonzero[0, :])

(4000, 64)
[1. 1. 1. 1. 1. 1. 3. 1. 3. 1. 4. 1. 1. 4. 3. 4. 4. 4. 4. 1. 1. 1. 4. 1.
 1. 1. 1. 1. 1. 1. 3. 3. 3. 4. 4. 4. 4. 1. 1. 3. 1. 4. 1. 4. 4. 3. 1. 3.
 3. 1. 4. 3. 3. 1. 1. 1. 4. 1. 4. 1. 4. 4. 1. 1.]


In [8]:
val_nonzero = np.ndarray(shape=(val_size, dim))
for row in range(val_2D.shape[0]):
    mat = val_2D[row].flatten()
    mat = mat[np.where(mat != 0)]
    val_nonzero[row, :] = mat[0:dim]
print(val_nonzero.shape)
print(val_nonzero[0, :])

(1000, 64)
[3. 3. 1. 4. 4. 2. 1. 4. 3. 1. 4. 4. 3. 2. 1. 2. 2. 4. 4. 3. 1. 3. 1. 1.
 1. 4. 4. 4. 2. 1. 3. 1. 1. 4. 4. 1. 3. 2. 1. 1. 3. 4. 1. 3. 1. 4. 1. 4.
 2. 1. 2. 1. 1. 1. 4. 1. 1. 4. 3. 2. 4. 4. 4. 1.]


In [9]:
lt_onehot = to_categorical(label_training) # make one hot vectors
lv_onehot = to_categorical(label_validation)
ltunique = np.unique(label_training).astype(int)
lvunique = np.unique(label_validation).astype(int)

In [10]:
print(ltunique)
print(lvunique)

[  50   83  200  524 1500]
[  50   83  200  524 1500]


In [11]:
lt_onehot = lt_onehot[:, ltunique]
lv_onehot = lv_onehot[:, lvunique]

In [12]:
print(lt_onehot.shape)
print(lv_onehot.shape)
print(lt[0])
print(lt_onehot[0])
print(lv[0])
print(lv_onehot[0])

(4000, 5)
(1000, 5)
200.0
[0. 0. 1. 0. 0.]
1500.0
[0. 0. 0. 0. 1.]


In [13]:
np.savez('hea_dataset.npz', train=train_nonzero, val=val_nonzero, train4D = trainset, val4D = valset, ltoh=lt_onehot, lvoh=lv_onehot, labtrain=label_training, labval=label_validation)